# EDA 및 시각화

In [167]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

In [8]:
suwon_time_flow_pop = pd.read_csv('4.수원시_시간대별_유동인구(2020).csv')
suwon_sex_age_flow_pop = pd.read_csv('5.수원시_성연령별_유동인구(2020).csv')
suwon_day_flow_pop = pd.read_csv('6.수원시_요일별_유동인구(2020).csv')
suwon_homeless = pd.read_csv('13.수원시_노숙인자활시설현황.csv')
suwon_child_welfare = pd.read_csv('14.수원시_아동복지시설현황.csv')

### 경계 매핑함수
- 격자 매핑함수
- 경찰서 경계 매핑함수

In [13]:
suwon_grid = gpd.read_file('32.수원시_표준격자(250M_x_250M).geojson')
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
# 데이터 불러오기

1) 격자 경계 매핑함수

In [132]:
def point_grid(data_df,suwon_grid,col):
    df = data_df.copy()
    if "geometry" in df.columns:
        pass
    else:
        epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
        df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
        df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    gid_result = gpd.sjoin(suwon_grid, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    # 매핑
    
    #gid_result_groupby = gid_result.groupby("gid").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    # gid로 groupby
    
    return gid_result

In [38]:
result_14 = point_grid(suwon_child_welfare,suwon_grid,"시설명")

In [44]:
result_14.head(3)

,gid,geometry,시설명,시설종류,lat,lon,수용인원
110,다사50ba21ab,"MULTIPOLYGON (((126.94155 37.28887, 126.94154 ...",참사랑,아동공동생활가정,37.289511,126.944004,6
371,다사52bb23aa,"MULTIPOLYGON (((126.96682 37.30476, 126.96681 ...",희망넝쿨,아동공동생활가정,37.306721,126.968909,6
461,다사53ab22bb,"MULTIPOLYGON (((126.97248 37.30254, 126.97247 ...",나섬의집,아동공동생활가정,37.302627,126.975104,7


In [45]:
result_14_groupby.head(3)

,gid,cnt
0,다사50ba21ab,1
1,다사52bb23aa,1
2,다사53ab22bb,1


1-1) 격자 경계 매핑 뒤 시각화 함수

2) 경찰서 경계 매핑함수

In [126]:
def point_police(data_df,others_police_area,col):
    df = data_df.copy()
    if "geometry" in df.columns:
        pass
    else:
        epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
        df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
        df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    others_police_area.columns = ["관할경찰서","관할경찰청","geometry"]
    
    police_result = gpd.sjoin(others_police_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    # 매핑
    
    if col == None:
        return police_result
    else:
        police_result_groupby = police_result.groupby("관할경찰서").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
        # 관할경찰서별로 groupby

        return police_result, police_result_groupby

#### 14.수원시 아동복지시설현황
- 비공개처리된 곳 제거하기

In [4]:
drop = ['road_addr', 'jibun_addr', 'phone_num', 'manage_agency', 'base_date']

Index(['fac_nm', 'fac_type', 'road_addr', 'jibun_addr', 'lat', 'lon',
       'phone_num', 'capacity', 'manage_agency', 'base_date'],
      dtype='object')

In [11]:
suwon_child_welfare = pd.read_csv('14.수원시_아동복지시설현황.csv')[['fac_nm', 'fac_type','lat', 'lon','capacity']]
# 열 선택

suwon_child_welfare.columns = ["시설명","시설종류","lat","lon","수용인원"]
# 열 이름 변경

suwon_child_welfare = suwon_child_welfare[suwon_child_welfare["lat"].notnull()]
# null 데이터 삭제

1) 14번 데이터 경찰서 경계로 매핑하기

In [51]:
result_14, result_14_groupby = point_police_df(suwon_child_welfare,others_police_area,"시설명")
result_14_groupby

,관할경찰서,cnt
0,수원남부경찰서,2
1,수원서부경찰서,3
2,수원중부경찰서,9


In [52]:
result_14.head(2)

,관할경찰서,관할경찰청,geometry,시설명,시설종류,lat,lon,수용인원
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",엘림그룹홈,아동공동생활가정,37.245276,127.029782,7
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",수원나자렛집,아동양육시설,37.249787,127.078692,15


2) 14번 데이터 격자 경계로 매핑하기

In [83]:
result_14_gird = point_grid(suwon_child_welfare,suwon_grid,"시설명")
result_14_gird.head(2)

,gid,geometry,시설명,시설종류,lat,lon,수용인원
110,다사50ba21ab,"MULTIPOLYGON (((126.94155 37.28887, 126.94154 ...",참사랑,아동공동생활가정,37.289511,126.944004,6
371,다사52bb23aa,"MULTIPOLYGON (((126.96682 37.30476, 126.96681 ...",희망넝쿨,아동공동생활가정,37.306721,126.968909,6


#### 13.수원시_노숙인자활시설현황
- 비공개처리된 곳 제거하기

In [55]:
drop = ['road_addr', 'jibun_addr','phone_num', 'base_date']

In [63]:
suwon_homeless = pd.read_csv('13.수원시_노숙인자활시설현황.csv')[['fac_nm','lat', 'lon', 'capacity']]
# 열 선택

suwon_homeless.columns = ["시설명","lat","lon","수용인원"]
# 열 이름 변경

1) 13번 데이터 경찰서 경계로 매핑하기

In [70]:
result_13_police, result_13_groupby_police = point_police_df(suwon_homeless,others_police_area,"시설명")
result_13_groupby_police

,관할경찰서,cnt
0,수원남부경찰서,1
1,수원서부경찰서,2
2,수원중부경찰서,1


In [68]:
result_13_police

,관할경찰서,관할경찰청,geometry,시설명,lat,lon,수용인원
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",새희망의집,37.272825,127.024826,20
29,수원서부경찰서,경기남부청,"MULTIPOLYGON (((126.99699 37.22926, 126.99698 ...",희망의쉼터,37.258268,126.991822,20
29,수원서부경찰서,경기남부청,"MULTIPOLYGON (((126.99699 37.22926, 126.99698 ...",마중물비전센터,37.273631,127.013395,20
30,수원중부경찰서,경기남부청,"MULTIPOLYGON (((127.03177 37.33654, 127.03176 ...",해뜨는집,37.295702,126.998741,20


2) 13번 데이터 격자 경계로 매핑하기

In [77]:
result_13_gird, result_13_groupby_gird = point_grid(suwon_homeless,suwon_grid,"시설명")
result_13_gird

,gid,geometry,시설명,lat,lon,수용인원
708,다사54bb17bb,"MULTIPOLYGON (((126.98971 37.25754, 126.98970 ...",희망의쉼터,37.258268,126.991822,20
872,다사55ba21bb,"MULTIPOLYGON (((126.99793 37.29363, 126.99792 ...",해뜨는집,37.295702,126.998741,20
1120,다사56bb19ba,"MULTIPOLYGON (((127.01216 37.27341, 127.01215 ...",마중물비전센터,37.273631,127.013395,20
1345,다사57bb19ab,"MULTIPOLYGON (((127.02346 37.27120, 127.02344 ...",새희망의집,37.272825,127.024826,20


#### 6.수원시_요일별_유동인구(2020)

In [ ]:
suwon_time_flow_pop = pd.read_csv('4.수원시_시간대별_유동인구(2020).csv')
suwon_sex_age_flow_pop = pd.read_csv('5.수원시_성연령별_유동인구(2020).csv')
suwon_day_flow_pop = pd.read_csv('6.수원시_요일별_유동인구(2020).csv')

In [105]:
suwon_day_flow_pop = pd.read_csv('6.수원시_요일별_유동인구(2020).csv')
# 데이터 읽어오기

suwon_day_flow_pop.columns = ["기준년월","월요일유동인구","화요일유동인구","수요일유동인구","목요일유동인구",
                             "금요일유동인구","토요일유동인구","일요일유동인구","lon","lat"]
# 열 이름 설정

suwon_day_flow_pop["년"] = suwon_day_flow_pop["기준년월"].apply(lambda x : int(str(x)[:4]))
suwon_day_flow_pop["월"] = suwon_day_flow_pop["기준년월"].apply(lambda x : int(str(x)[4:]))
suwon_day_flow_pop = suwon_day_flow_pop.drop(["기준년월"],axis=1)
# 기준년월 데이터 파싱

1) 6번 데이터 경찰서 경계로 매핑하기

result_6_police = point_police(suwon_day_flow_pop,others_police_area,None)

In [137]:
result_6_police.head(2)

,관할경찰서,관할경찰청,geometry,기준년월,월요일유동인구,화요일유동인구,수요일유동인구,목요일유동인구,금요일유동인구,토요일유동인구,일요일유동인구,lon,lat,년,월
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",202008,369.12,372.53,381.89,375.85,405.77,358.32,289.01,127.023553,37.227299,2020,8
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",202005,418.25,403.01,439.93,432.82,452.82,404.11,324.94,127.023553,37.227299,2020,5


2) 6번 데이터 격자 경계로 매핑하기

In [133]:
result_6_grid = point_grid(suwon_day_flow_pop,suwon_grid,None)

In [135]:
result_6_grid.head(3)

,gid,geometry,기준년월,월요일유동인구,화요일유동인구,수요일유동인구,목요일유동인구,금요일유동인구,토요일유동인구,일요일유동인구,lon,lat,년,월
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",202005,0.11,0.16,0.12,0.12,0.14,0.26,0.26,126.927402,37.271492,2020,5
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",202011,7.82,7.20,7.12,7.03,7.73,11.58,10.79,126.927402,37.271492,2020,11
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",202003,0.12,0.14,0.12,0.11,0.12,0.28,0.28,126.927402,37.271492,2020,3


#### 5.수원시_성연령별_유동인구(2020)

In [140]:
suwon_sex_age_flow_pop = pd.read_csv('5.수원시_성연령별_유동인구(2020).csv')
# 데이터 읽어오기

suwon_sex_age_flow_pop.columns = ["기준년월","남자유동인구수_10대_수치","남자유동인구수_20대_수치","남자유동인구수_30대_수치",
                                 "남자유동인구수_40대_수치","남자유동인구수_50대_수치","남자유동인구수_60대이상_수치",
                                 "여자유동인구수_10대_수치","여자유동인구수_20대_수치","여자유동인구수_30대_수치",
                                 "여자유동인구수_40대_수치","여자유동인구수_50대_수치","여자유동인구수_60대이상_수치","lon","lat"]
# 열 이름 설정

suwon_sex_age_flow_pop["년"] = suwon_sex_age_flow_pop["기준년월"].apply(lambda x : int(str(x)[:4]))
suwon_sex_age_flow_pop["월"] = suwon_sex_age_flow_pop["기준년월"].apply(lambda x : int(str(x)[4:]))
suwon_sex_age_flow_pop = suwon_sex_age_flow_pop.drop(["기준년월"],axis=1)
# 기준년월 데이터 파싱

In [142]:
suwon_sex_age_flow_pop.head(2)

,남자유동인구수_10대_수치,남자유동인구수_20대_수치,남자유동인구수_30대_수치,남자유동인구수_40대_수치,남자유동인구수_50대_수치,남자유동인구수_60대이상_수치,여자유동인구수_10대_수치,여자유동인구수_20대_수치,여자유동인구수_30대_수치,여자유동인구수_40대_수치,여자유동인구수_50대_수치,여자유동인구수_60대이상_수치,lon,lat,년,월
0,0.00,0.02,0.04,0.04,0.04,0.02,0.00,0.00,0.04,0.04,0.04,0.02,126.929668,37.270151,2020,1
1,0.06,0.08,0.06,0.10,0.09,0.08,0.06,0.06,0.07,0.12,0.09,0.10,126.930229,37.270604,2020,1


1) 5번 데이터 경찰서 경계로 매핑하기

In [143]:
result_5_police = point_police(suwon_sex_age_flow_pop,others_police_area,None)

In [144]:
result_5_police.head(2)

,관할경찰서,관할경찰청,geometry,남자유동인구수_10대_수치,남자유동인구수_20대_수치,남자유동인구수_30대_수치,남자유동인구수_40대_수치,남자유동인구수_50대_수치,남자유동인구수_60대이상_수치,여자유동인구수_10대_수치,여자유동인구수_20대_수치,여자유동인구수_30대_수치,여자유동인구수_40대_수치,여자유동인구수_50대_수치,여자유동인구수_60대이상_수치,lon,lat,년,월
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",0.85,1.22,2.65,3.65,3.09,1.97,0.77,0.82,1.47,2.10,1.65,1.14,127.053961,37.231924,2020,5
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",0.51,1.02,2.72,3.38,2.94,1.90,0.57,0.68,1.33,1.86,1.44,0.99,127.053961,37.231924,2020,9


2) 5번 데이터 격자 경계로 매핑하기

In [145]:
result_5_grid = point_grid(suwon_sex_age_flow_pop,suwon_grid,None)

In [146]:
result_5_grid.head(2)

,gid,geometry,남자유동인구수_10대_수치,남자유동인구수_20대_수치,남자유동인구수_30대_수치,남자유동인구수_40대_수치,남자유동인구수_50대_수치,남자유동인구수_60대이상_수치,여자유동인구수_10대_수치,여자유동인구수_20대_수치,여자유동인구수_30대_수치,여자유동인구수_40대_수치,여자유동인구수_50대_수치,여자유동인구수_60대이상_수치,lon,lat,년,월
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0.0,0.0,0.02,0.04,0.04,0.02,0.0,0.0,0.0,0.04,0.04,0.0,126.927402,37.271492,2020,5
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0.0,0.0,0.00,0.02,0.04,0.02,0.0,0.0,0.0,0.04,0.02,0.0,126.927402,37.271492,2020,6


#### 4.수원시_시간대별_유동인구(2020)

In [158]:
suwon_time_flow_pop = pd.read_csv('4.수원시_시간대별_유동인구(2020).csv')

suwon_time_flow_pop.columns = ["기준년월",'0시기준_측정정보', '1시기준_측정정보', '2시기준_측정정보', '3시기준_측정정보', 
                               '4시기준_측정정보', '5시기준_측정정보', '6시기준_측정정보', '7시기준_측정정보', '8시기준_측정정보', 
                               '9시기준_측정정보', '10시기준_측정정보', '11시기준_측정정보', '12시기준_측정정보', '13시기준_측정정보',
                               '14시기준_측정정보', '15시기준_측정정보', '16시기준_측정정보', '17시기준_측정정보', '18시기준_측정정보',
                               '19시기준_측정정보', '20시기준_측정정보', '21시기준_측정정보', '22시기준_측정정보', '23시기준_측정정보',
                               "lon","lat"]
# 열 이름 설정

suwon_time_flow_pop["년"] = suwon_time_flow_pop["기준년월"].apply(lambda x : int(str(x)[:4]))
suwon_time_flow_pop["월"] = suwon_time_flow_pop["기준년월"].apply(lambda x : int(str(x)[4:]))
suwon_time_flow_pop = suwon_time_flow_pop.drop(["기준년월"],axis=1)
# 기준년월 데이터 파싱

1) 4번 데이터 경찰서 경계로 매핑하기

In [160]:
result_4_police = point_police(suwon_time_flow_pop,others_police_area,None)

In [161]:
result_4_police.head(2)

,관할경찰서,관할경찰청,geometry,0시기준_측정정보,1시기준_측정정보,2시기준_측정정보,3시기준_측정정보,4시기준_측정정보,5시기준_측정정보,6시기준_측정정보,...,18시기준_측정정보,19시기준_측정정보,20시기준_측정정보,21시기준_측정정보,22시기준_측정정보,23시기준_측정정보,lon,lat,년,월
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",0.51,0.60,0.35,0.32,0.28,0.41,1.01,...,2.37,1.75,1.45,1.27,1.02,0.78,127.053961,37.231924,2020,11
18,수원남부경찰서,경기남부청,"MULTIPOLYGON (((127.03963 37.32102, 127.03969 ...",0.42,0.26,0.21,0.18,0.18,0.28,0.81,...,1.89,1.52,1.15,0.92,0.95,0.87,127.053961,37.231924,2020,5


In [162]:
print(result_4_police.shape,suwon_time_flow_pop.shape)

(414528, 31) (416080, 28)


2) 4번 데이터 격자 경계로 매핑하기

In [164]:
result_4_grid = point_grid(suwon_time_flow_pop,suwon_grid,None)

In [165]:
result_4_grid.head(2)

,gid,geometry,0시기준_측정정보,1시기준_측정정보,2시기준_측정정보,3시기준_측정정보,4시기준_측정정보,5시기준_측정정보,6시기준_측정정보,7시기준_측정정보,...,18시기준_측정정보,19시기준_측정정보,20시기준_측정정보,21시기준_측정정보,22시기준_측정정보,23시기준_측정정보,lon,lat,년,월
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0.04,0.04,0.02,0.02,0.02,0.02,0.05,0.04,...,0.07,0.05,0.05,0.04,0.04,0.04,126.927402,37.271492,2020,7
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.02,0.02,0.00,0.00,0.00,0.00,126.927402,37.271492,2020,8


In [166]:
print(result_4_grid.shape,suwon_time_flow_pop.shape)

(415814, 30) (416080, 28)


### 4,5,6번 격자 데이터 합치기
- suwon_time_flow_pop = pd.read_csv('4.수원시_시간대별_유동인구(2020).csv')
- suwon_sex_age_flow_pop = pd.read_csv('5.수원시_성연령별_유동인구(2020).csv')
- suwon_day_flow_pop = pd.read_csv('6.수원시_요일별_유동인구(2020).csv')

In [174]:
concat1 = pd.merge(suwon_time_flow_pop,suwon_sex_age_flow_pop,on = ["lon","lat","년","월"],how="inner")
concat2 = pd.merge(suwon_day_flow_pop,concat1,on = ["lon","lat","년","월"],how="inner")

In [176]:
concat2.columns

Index(['기준년월', '월요일유동인구', '화요일유동인구', '수요일유동인구', '목요일유동인구', '금요일유동인구',
       '토요일유동인구', '일요일유동인구', 'lon', 'lat', '년', '월', '0시기준_측정정보', '1시기준_측정정보',
       '2시기준_측정정보', '3시기준_측정정보', '4시기준_측정정보', '5시기준_측정정보', '6시기준_측정정보',
       '7시기준_측정정보', '8시기준_측정정보', '9시기준_측정정보', '10시기준_측정정보', '11시기준_측정정보',
       '12시기준_측정정보', '13시기준_측정정보', '14시기준_측정정보', '15시기준_측정정보', '16시기준_측정정보',
       '17시기준_측정정보', '18시기준_측정정보', '19시기준_측정정보', '20시기준_측정정보', '21시기준_측정정보',
       '22시기준_측정정보', '23시기준_측정정보', '남자유동인구수_10대_수치', '남자유동인구수_20대_수치',
       '남자유동인구수_30대_수치', '남자유동인구수_40대_수치', '남자유동인구수_50대_수치',
       '남자유동인구수_60대이상_수치', '여자유동인구수_10대_수치', '여자유동인구수_20대_수치',
       '여자유동인구수_30대_수치', '여자유동인구수_40대_수치', '여자유동인구수_50대_수치',
       '여자유동인구수_60대이상_수치'],
      dtype='object')